# Summary
This notebook provides a basic analysis of the comments including example comments, how many comments are from templates, and how many unique comments exist.

In [19]:
import pandas
import numpy as np
import folium
from folium import plugins

The comments are loaded in from the Data Cleanup notebook.

In [20]:
data = pandas.read_json('./data/comments_cleaned.json', orient='records', dtype='false')

Below is a print out of the pandas dataframe schema and first comment.

In [39]:
print("Loaded %d comments." % len(data))
display(data[:1])

Loaded 12225 comments.


,doc.attachment_download,doc.attachment_download -href,doc.attachment_name,doc.category,doc.city,doc.comment_body,doc.country,doc.name,doc.state,doc.zip
0,None,None,None,None,United States,"Dear Assistant General Counsel Hilary Malawer,...",Parent/Relative,Heather Hirsch,MN,55016


A sample of the comment_body in the dataset.

In [22]:
with pandas.option_context('display.max_colwidth', 5000):
    display(data[['doc.comment_body']].sample(5))

,doc.comment_body
5791,"Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity.\nI urge the Department to keep in its current form 34 C.F.R. pts. 1 thru 1299 , which include regulations governing the Secretary and the offices for Civil Rights; Elementary and Secondary Education; Special Education and Rehabilitative Services; Career, Technical, and Adult Education; Post-Secondary Education; Educational Research and Improvement; and the National Council on Disability. \n\nI also urge the Department to preserve all current significant guidance documents, including guidance on sexual, racial, and disability-based harassment (including guidance on sexual violence); access to athletic opportunities; gender equity in career and technical education; single-sex schools; equal access to educational resources; nondiscriminatory school discipline; racial diversity programs; the rights of students with disabilities in charter schools; restraint and seclusion of students with disabilities; and the rights of English language learners. I urge you to keep current regulations and guidance in place, and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive.\n\nSincerely,\nMatthew Faulkner\n Shawnee, KS 66203"
6461,"Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity.\nI urge the Department to keep in its current form 34 C.F.R. pts. 1 thru 1299 , which include regulations governing the Secretary and the offices for Civil Rights; Elementary and Secondary Education; Special Education and Rehabilitative Services; Career, Technical, and Adult Education; Post-Secondary Education; Educational Research and Improvement; and the National Council on Disability. \n\nI also urge the Department to preserve all current significant guidance documents, including guidance on sexual, racial, and disability-based harassment (including guidance on sexual violence); access to athletic opportunities; gender equity in career and technical education; single-sex schools; equal access to educational resources; nondiscriminatory school discipline; racial diversity programs; the rights of students with disabilities in charter schools; restraint and seclusion of students with disabilities; and the rights of English language learners. I urge you to keep current regulations and guidance in place, and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive.\n\nSincerely,\nKathie Noga\n Minneapolis, MN 55404"
6358,"Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity.\nI urge t

Below is a list of common strings to use in pattern matching and see how many similar comments exist. This list is handpicked and the matching is based on the one conducted [here](https://github.com/j2kao/fcc_nn_research/blob/master/proc_17_108_analysis_01_level_0_manual_tagging.ipynb) for FCC comments

In [23]:
common_strings = "Dear Assistant General Counsel Hilary Malawer,\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary"
common_string2 = "Dear Assistant General Counsel Hilary Malawer,\n\nCurrent federal regulations and guidance help all studentsregardless of sex, race, color, sexual orientation,"

In [35]:
dup_removed = data.copy()

# Find rows containing the common strings
invalid_indexes = []
invalid_indexes.extend(data[data['doc.comment_body'].str.contains(common_strings, case=False, na=False)].index.values)
invalid_indexes.extend(data[data['doc.comment_body'].str.contains(common_string2, case=False, na=False)].index.values)

dup_removed.drop(invalid_indexes, inplace=True)
dup_removed = dup_removed.reset_index(drop=True)

# Saving the dataset with duplicates removed for later use
dup_removed.to_json('./data/comments_duplicates_removed.json')

print("Total comments after removing duplicates: ", len(dup_removed))

Total comments after removing duplicates:  2868


Another look at the dataset, now with the most common comments removed.

In [25]:
with pandas.option_context('display.max_colwidth', 5000):
    display(dup_removed[['doc.comment_body']].sample(50))

,doc.comment_body
7808,"Department of Education ED/OII, Completed Actions, Charter Schools Grants to SEAs, 1855-AA12""Gainful Employment"" rule to prepare students for employment in a recognized occupation Income-driven ""pay as you earn"" program, Race to the Top, Every Student Succeeds Act (ESSA)"
11022,"Subject; <WIOA> In accordance with Executive Order 13777, Enforcing the Regulatory Reform Agenda, the dept of Education is seeking input on regulations THAT MAY BE APPROPRIATE FOR REPEAL, REPLACEMENT, OR MODIFICATION.\n\n We request that the homemaker rules must be repealed because a person that has LOST their eye sight must have the chance to be trained to be Independent and be able to live on their own. The cost of this training is much less than having to put this person in a facility because they cant take care of themselves. The cost of this training would be a fraction to the Department of Rehabilitation compared to the cost to Medicare, Medi-cade, social security at an early age, or just their care being charged to one of the disability programs.\n\n This training is for maybe 40 weeks and then the person would be able to live independently for the REST of their LIFE. Wouldnt you agree that would be a better quality of life and much more of a better way to spend TAX PAYERS money than just putting then in a facility that they couldnt pay the fee to stay the for the rest of the life with no way to HAVE A CHANCE TO BE A PRODUCTIVE PERSON AGAIN. Just because they go thru the training as a HOMEMAKER is NOT the reason or mean things could change after they get this training for INDEPENDENCE \n\n We really need you to repeal order 13777 for the reasons I have stated plus many more reasons. Maybe they lost their sight serving our Country, State, or Community, or they lost it doing the job that took care of the family.\n\n Thank you Leonard A. Blottin President of the Board of the SAN DIEGO CENTER FOR THE BLIND. \n\n A non profit 501C-3 Organization.\n"
9116,protect victims of college campus sexual assault rather than pretend a serious problem doesn't exist!
4646,"Dear Assistant General Counsel Hilary Malawer,\n\nLet's stop selling everything to greedy people.\n\nAll Department of Education civil rights regulations and guidance documents are important and necessary. Far from being burdensome, current civil rights rules and regulations benefit schools and students by providing a clear framework that, when followed, allow all students an equal opportunity to learn in a safe and welcoming environment regardless of sex, race, color, national origin, disability status, English proficiency, sexual orientation, or gender identity.\nI urge the Department to keep in its current form 34 C.F.R. pts. 1 thru 1299 , which include regulations governing the Secretary and the offices for Civil Rights; Elementary and Secondary Education; Special Education and Rehabilitative Services; Career, Technical, and Adult Education; Post-Secondary Education; Educational Research and Improvement; and the National Council on Disability. \n\nI also urge the Department to preserve all current significant guidance documents, including guidance on sexual, racial, and disability-based harassment (including guidance on sexual violence); access to athletic opportunities; gender equity in career and technical education; single-sex schools; equal access to educational resources; nondiscriminatory school discipline; racial diversity programs; the rights of students with disabilities in charter schools; restraint and seclusion of students with disabilities; and the rights of English language learners. I urge you to keep current regulations and guidance in place, and to continue enforcing these critical civil rights laws so that all students have an equal opportunity to learn and thrive.\n\nSincerely,\nJohn Wilson\n Magdalena, NM 87825"
9725,"The mission of the Department of education is to educate the people of our land. Choosing to educate only some of the people do

Creating a dataframe of states and zip codes from the cleaned comments dataset (dupicate comments intact) to use in a map.

In [26]:
map_data = data[['doc.state', 'doc.zip']]

# Remove any null values
map_data.dropna(inplace=True)

/mnt/c/Users/nish/projects/titleix/env/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Generating a heat map visualization by state of where comments came from along with listing the states from which the most comments were sent.

In [27]:
state_data = map_data.groupby(['doc.state']).size().reset_index(name='duplicate_count')
state_data = state_data.sort_values('duplicate_count', ascending=False).reset_index(drop=True)

zip_data = map_data.groupby(['doc.zip']).size().reset_index(name='duplicate_count')
zip_data = zip_data.sort_values('duplicate_count', ascending=False).reset_index(drop=True)

display(state_data[:5], zip_data[:5])

,doc.state,duplicate_count
0,CA,1464
1,NY,727
2,WA,441
3,FL,423
4,PA,383


,doc.zip,duplicate_count
0,10025,18
1,11215,18
2,10011,15
3,10023,13
4,95060,12


In [28]:
state_geo = './resources/us-states.json'

map = folium.Map(location=[48, -102], zoom_start=3)
map.choropleth(
    geo_data=state_geo,
    name='map',
    data=state_data,
    columns=['doc.state', 'duplicate_count'],
    key_on='feature.id',
    fill_color='BuPu',
    fill_opacity=0.9,
    line_opacity=0.1
)
folium.LayerControl().add_to(map)

map

Resources:
https://github.com/j2kao/fcc_nn_research/blob/master/proc_17_108_analysis_01_level_0_manual_tagging.ipynb